In [1]:
# 1. Establish a connection between Python and the Sakila database.
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy import create_engine

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'
connection_string = f'{driver}://{user}:{password}@{ip}'

Your password:········


In [2]:
db_connection = sa.create_engine(connection_string)
result = db_connection.execute('SELECT * FROM sakila.store')
for row in result:
    print(row)

(1, 1, 1, datetime.datetime(2006, 2, 15, 4, 57, 12))
(2, 2, 2, datetime.datetime(2006, 2, 15, 4, 57, 12))


In [32]:
# 2. Write a Python function called rentals_month that retrieves rental data for a given month and year

def rentals_month(engine, month, year):
    query = "SELECT * from sakila.rental \
                WHERE date_format(rental_date, '%%m') = " + str(month) + \
                " AND date_format(rental_date, '%%Y') = " + str(year)
    rental = pd.read_sql_query(query, engine)
    return rental, month, year

df, m, y = rentals_month(db_connection, 5, 2005)
df
    
    

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,month,weekday,daytype
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,5,1,workday
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,5,1,workday
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,5,1,workday
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,5,1,workday
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,5,1,workday
...,...,...,...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53,5,1,workday
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53,5,1,workday
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53,5,1,workday
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53,5,1,workday


In [33]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

def rental_count_month(df, month, year):
    df = df.groupby('customer_id')[['rental_id']].count()
    df.rename(columns={'rental_id':f'rentals_{month}_{year}'}, inplace=True)
    return df

df1 = rental_count_month(df, m, y)
df1

,rentals_5_2005
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [70]:
# 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

def compare_rentals(df1, df2):
    df1.reset_index(inplace=True)
    df2.reset_index(inplace=True)
    df3 = df1.merge(df2, on='customer_id', how='outer')
    df3.fillna(0,inplace=True)
    df3['difference'] = df3[df3.columns[1]] - df3[df3.columns[2]]
    return df3

df1 = rental_count_month(*rentals_month(db_connection, 5, 2005))
df2 = rental_count_month(*rentals_month(db_connection, 6, 2005))
df3 = compare_rentals(df1, df2)
df3

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,5,3.0,5.0,-2.0
4,6,3.0,4.0,-1.0
...,...,...,...,...
593,583,0.0,6.0,-6.0
594,585,0.0,4.0,-4.0
595,591,0.0,3.0,-3.0
596,592,0.0,5.0,-5.0
